# Haar System

In [80]:
import numpy as np

def approximation(array):
    return array[:int(len(array)/2)]

def details(array):
    return array[int(len(array)/2):]

def i_num(array):
    return int(np.ceil(np.log2(len(array))))

root2 = np.sqrt(2)
s = np.array([32, 32, 16, 8, 24, 16, 64, 32])

### For METHOD


In [66]:
def direct_wavelet(array,output):
    N, N2, = len(array), int(len(array)/2)
    for i in range(0,N,2):
        output[int(i/2)] = (array[i] + array[i+1])/root2
        output[int(i/2 + N2)] = (array[i] - array[i+1])/root2
    return output

def inverse_wavelet(output,signal):
    N, N2,  = len(output), int(len(output)/2)
    for i in range(0,N,2):
        signal[i] = (output[int(i/2)] + output[int(i/2 + N2)])/root2
        signal[i+1] = (output[int(i/2)] - output[int(i/2 + N2)])/root2
    return signal

# DIRECT WEVELET TRANSFORM --> dwt
dwt = np.empty(len(s))
dwt = direct_wavelet(s,dwt)
print("\nApproximation coefficients (A1):", approximation(dwt))
print("Detail coefficients (D1):", details(dwt))

# INVERSE WEVELET TRANSFORM 
rest_s = np.empty(len(s))
rest_s = inverse_wavelet(dwt,rest_s)
print("\nReconstructed Signal:", rest_s)
print("Original Signal:", s)


Approximation coefficients (A1): [45.254834   16.97056275 28.28427125 67.88225099]
Detail coefficients (D1): [ 0.          5.65685425  5.65685425 22.627417  ]

Reconstructed Signal: [32. 32. 16.  8. 24. 16. 64. 32.]
Original Signal: [32 32 16  8 24 16 64 32]


In [82]:
dwt = np.empty(len(s))
dwt = direct_wavelet(s, dwt)

for i in range(1, i_num(s)+1):
    print(f"\nApproximation coefficients (A{i}):", approximation(dwt))
    print(f"Detail coefficients (D{i}):", details(dwt))
    if i>1:
        dwt_temp = np.empty(len(approximation(dwt)))
        dwt_temp = direct_wavelet(approximation(dwt), dwt_temp)
        dwt = dwt_temp


Approximation coefficients (A1): [45.254834   16.97056275 28.28427125 67.88225099]
Detail coefficients (D1): [ 0.          5.65685425  5.65685425 22.627417  ]

Approximation coefficients (A2): [45.254834   16.97056275 28.28427125 67.88225099]
Detail coefficients (D2): [ 0.          5.65685425  5.65685425 22.627417  ]

Approximation coefficients (A3): [44. 68.]
Detail coefficients (D3): [ 20. -28.]
